# Reti neurali ricorrenti

Nel modulo precedente, abbiamo utilizzato rappresentazioni semantiche ricche del testo e un semplice classificatore lineare sopra gli embeddings. Questa architettura cattura il significato aggregato delle parole in una frase, ma non tiene conto dell'**ordine** delle parole, poiché l'operazione di aggregazione sugli embeddings elimina questa informazione dal testo originale. Poiché questi modelli non sono in grado di modellare l'ordine delle parole, non possono risolvere compiti più complessi o ambigui come la generazione di testo o la risposta alle domande.

Per catturare il significato della sequenza di testo, dobbiamo utilizzare un'altra architettura di rete neurale, chiamata **rete neurale ricorrente**, o RNN. In una RNN, passiamo la nostra frase attraverso la rete un simbolo alla volta, e la rete produce uno **stato**, che poi passiamo nuovamente alla rete insieme al simbolo successivo.

Dato la sequenza di token $X_0,\dots,X_n$, la RNN crea una sequenza di blocchi di rete neurale e addestra questa sequenza end-to-end utilizzando la retropropagazione. Ogni blocco di rete prende una coppia $(X_i,S_i)$ come input e produce $S_{i+1}$ come risultato. Lo stato finale $S_n$ o l'output $X_n$ viene inviato a un classificatore lineare per produrre il risultato. Tutti i blocchi di rete condividono gli stessi pesi e vengono addestrati end-to-end con un unico passaggio di retropropagazione.

Poiché i vettori di stato $S_0,\dots,S_n$ vengono passati attraverso la rete, essa è in grado di apprendere le dipendenze sequenziali tra le parole. Ad esempio, quando la parola *not* appare da qualche parte nella sequenza, può imparare a negare determinati elementi all'interno del vettore di stato, risultando in una negazione.

> Poiché i pesi di tutti i blocchi RNN nell'immagine sono condivisi, la stessa immagine può essere rappresentata come un unico blocco (a destra) con un ciclo di feedback ricorrente, che passa lo stato di output della rete nuovamente all'input.

Vediamo come le reti neurali ricorrenti possono aiutarci a classificare il nostro dataset di notizie.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Classificatore RNN semplice

Nel caso di un RNN semplice, ogni unità ricorrente è una rete lineare semplice, che prende un vettore di input concatenato e un vettore di stato, e produce un nuovo vettore di stato. PyTorch rappresenta questa unità con la classe `RNNCell`, e una rete di tali celle - come livello `RNN`.

Per definire un classificatore RNN, applicheremo prima un livello di embedding per ridurre la dimensionalità del vocabolario di input, e poi aggiungeremo un livello RNN sopra di esso:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Nota:** Usiamo un livello di embedding non addestrato qui per semplicità, ma per ottenere risultati ancora migliori possiamo utilizzare un livello di embedding pre-addestrato con embedding Word2Vec o GloVe, come descritto nell'unità precedente. Per una migliore comprensione, potresti voler adattare questo codice per funzionare con embedding pre-addestrati.

Nel nostro caso, utilizzeremo un data loader con padding, quindi ogni batch avrà un certo numero di sequenze con padding della stessa lunghezza. Il livello RNN prenderà la sequenza di tensori di embedding e produrrà due output:  
* $x$ è una sequenza di output delle celle RNN a ogni passo  
* $h$ è lo stato nascosto finale per l'ultimo elemento della sequenza  

Applichiamo quindi un classificatore lineare completamente connesso per ottenere il numero di classi.

> **Nota:** Gli RNN sono piuttosto difficili da addestrare, perché una volta che le celle RNN vengono "srotolate" lungo la lunghezza della sequenza, il numero risultante di livelli coinvolti nella retropropagazione è piuttosto elevato. Pertanto, è necessario selezionare un tasso di apprendimento basso e addestrare la rete su un dataset più grande per ottenere buoni risultati. Può richiedere molto tempo, quindi è preferibile utilizzare una GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Uno dei principali problemi delle RNN classiche è il cosiddetto problema dei **vanishing gradients**. Poiché le RNN vengono addestrate end-to-end in un unico passaggio di back-propagation, hanno difficoltà a propagare l'errore ai primi strati della rete, e di conseguenza la rete non riesce a imparare le relazioni tra token distanti. Uno dei modi per evitare questo problema è introdurre una **gestione esplicita dello stato** utilizzando i cosiddetti **gates**. Esistono due architetture più conosciute di questo tipo: **Long Short Term Memory** (LSTM) e **Gated Relay Unit** (GRU).

![Immagine che mostra un esempio di cella LSTM](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

La rete LSTM è organizzata in modo simile a una RNN, ma ci sono due stati che vengono passati da uno strato all'altro: lo stato effettivo $c$ e il vettore nascosto $h$. In ogni unità, il vettore nascosto $h_i$ viene concatenato con l'input $x_i$, e insieme controllano cosa accade allo stato $c$ tramite i **gates**. Ogni gate è una rete neurale con attivazione sigmoide (output nell'intervallo $[0,1]$), che può essere considerata come una maschera bitwise quando moltiplicata per il vettore di stato. I gates sono i seguenti (da sinistra a destra nell'immagine sopra):
* **forget gate** prende il vettore nascosto e determina quali componenti del vettore $c$ dobbiamo dimenticare e quali far passare.
* **input gate** prende alcune informazioni dall'input e dal vettore nascosto, e le inserisce nello stato.
* **output gate** trasforma lo stato tramite un layer lineare con attivazione $\tanh$, quindi seleziona alcune delle sue componenti utilizzando il vettore nascosto $h_i$ per produrre il nuovo stato $c_{i+1}$.

Le componenti dello stato $c$ possono essere considerate come dei flag che possono essere attivati o disattivati. Ad esempio, quando incontriamo il nome *Alice* in una sequenza, potremmo voler assumere che si riferisca a un personaggio femminile, e attivare il flag nello stato che indica la presenza di un sostantivo femminile nella frase. Quando successivamente incontriamo la frase *e Tom*, attiveremo il flag che indica la presenza di un sostantivo plurale. In questo modo, manipolando lo stato, possiamo teoricamente tenere traccia delle proprietà grammaticali delle parti della frase.

> **Nota**: Una risorsa eccellente per comprendere i dettagli interni delle LSTM è questo fantastico articolo [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) di Christopher Olah.

Sebbene la struttura interna di una cella LSTM possa sembrare complessa, PyTorch nasconde questa implementazione all'interno della classe `LSTMCell` e fornisce l'oggetto `LSTM` per rappresentare l'intero layer LSTM. Pertanto, l'implementazione di un classificatore LSTM sarà molto simile a quella della semplice RNN che abbiamo visto sopra:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Sequenze compattate

Nel nostro esempio, abbiamo dovuto riempire tutte le sequenze nel minibatch con vettori di zeri. Sebbene ciò comporti un certo spreco di memoria, con le RNN è ancora più critico il fatto che vengano create celle RNN aggiuntive per gli elementi di input riempiti, che partecipano all'addestramento ma non contengono informazioni significative. Sarebbe molto meglio addestrare la RNN solo sulla dimensione effettiva della sequenza.

Per fare ciò, in PyTorch viene introdotto un formato speciale per la memorizzazione delle sequenze riempite. Supponiamo di avere un minibatch di input riempito che appare così:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Qui 0 rappresenta i valori riempiti, e il vettore delle lunghezze effettive delle sequenze di input è `[5,3,1]`.

Per addestrare efficacemente una RNN con sequenze riempite, vogliamo iniziare l'addestramento del primo gruppo di celle RNN con un grande minibatch (`[1,6,9]`), ma poi terminare l'elaborazione della terza sequenza e continuare l'addestramento con minibatch più piccoli (`[2,7]`, `[3,8]`), e così via. Pertanto, una sequenza compattata è rappresentata come un unico vettore - nel nostro caso `[1,6,9,2,7,3,8,4,5]`, e un vettore delle lunghezze (`[5,3,1]`), dal quale possiamo facilmente ricostruire il minibatch originale riempito.

Per produrre una sequenza compattata, possiamo utilizzare la funzione `torch.nn.utils.rnn.pack_padded_sequence`. Tutti i livelli ricorrenti, inclusi RNN, LSTM e GRU, supportano le sequenze compattate come input e producono un output compattato, che può essere decodificato utilizzando `torch.nn.utils.rnn.pad_packed_sequence`.

Per poter produrre una sequenza compattata, dobbiamo passare il vettore delle lunghezze alla rete, e quindi abbiamo bisogno di una funzione diversa per preparare i minibatch:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

La rete effettiva sarebbe molto simile a `LSTMClassifier` sopra, ma il passaggio `forward` riceverà sia il minibatch con padding che il vettore delle lunghezze delle sequenze. Dopo aver calcolato l'embedding, calcoliamo la sequenza impacchettata, la passiamo al livello LSTM e poi scompattiamo il risultato.

> **Nota**: In realtà non utilizziamo il risultato scompattato `x`, perché usiamo l'output dei livelli nascosti nelle computazioni successive. Pertanto, possiamo rimuovere del tutto lo scompattamento da questo codice. Il motivo per cui lo includiamo qui è per permetterti di modificare facilmente questo codice, nel caso in cui dovessi aver bisogno di utilizzare l'output della rete in ulteriori computazioni.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Nota:** Potresti aver notato il parametro `use_pack_sequence` che passiamo alla funzione di training. Attualmente, la funzione `pack_padded_sequence` richiede che il tensore della sequenza di lunghezza sia sul dispositivo CPU e, di conseguenza, la funzione di training deve evitare di spostare i dati della sequenza di lunghezza su GPU durante il training. Puoi esaminare l'implementazione della funzione `train_emb` nel file [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Reti neurali ricorrenti bidirezionali e multilivello

Nei nostri esempi, tutte le reti ricorrenti operavano in una sola direzione, dall'inizio di una sequenza alla fine. Sembra naturale, perché somiglia al modo in cui leggiamo e ascoltiamo il parlato. Tuttavia, poiché in molti casi pratici abbiamo accesso casuale alla sequenza di input, potrebbe avere senso eseguire il calcolo ricorrente in entrambe le direzioni. Queste reti sono chiamate **RNN bidirezionali**, e possono essere create passando il parametro `bidirectional=True` al costruttore di RNN/LSTM/GRU.

Quando si lavora con una rete bidirezionale, avremo bisogno di due vettori di stato nascosto, uno per ciascuna direzione. PyTorch codifica questi vettori come un unico vettore di dimensione doppia, il che è piuttosto conveniente, perché normalmente si passa lo stato nascosto risultante a un livello lineare completamente connesso, e basta tenere conto di questo aumento di dimensione quando si crea il livello.

Una rete ricorrente, sia unidirezionale che bidirezionale, cattura determinati schemi all'interno di una sequenza e può memorizzarli nel vettore di stato o passarli all'output. Come con le reti convoluzionali, possiamo costruire un altro livello ricorrente sopra il primo per catturare schemi di livello superiore, costruiti a partire dagli schemi di basso livello estratti dal primo livello. Questo ci porta al concetto di **RNN multilivello**, che consiste in due o più reti ricorrenti, dove l'output del livello precedente viene passato al livello successivo come input.

![Immagine che mostra una rete RNN LSTM multilivello](../../../../../translated_images/it/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Immagine tratta da [questo fantastico articolo](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) di Fernando López*

PyTorch rende la costruzione di tali reti un compito semplice, perché basta passare il parametro `num_layers` al costruttore di RNN/LSTM/GRU per costruire automaticamente diversi livelli di ricorrenza. Questo significa anche che la dimensione del vettore di stato nascosto aumenterà proporzionalmente, e sarà necessario tenerne conto quando si gestisce l'output dei livelli ricorrenti.


## RNN per altri compiti

In questa unità, abbiamo visto che le RNN possono essere utilizzate per la classificazione di sequenze, ma in realtà possono gestire molti altri compiti, come la generazione di testo, la traduzione automatica e altro ancora. Considereremo questi compiti nella prossima unità.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
